In [0]:
from pyspark.sql import functions as f

In [0]:
# start date and end date
start_date = "2024-01-01"
end_date   = "2025-12-01" 

In [0]:
#generate one row for each month between start_date and end_date
df = (
    spark.sql(f"""
        SELECT explode(
            sequence(
                to_date('{start_date}'),
                to_date('{end_date}'),
                interval 1 month
            )
        ) AS month_start_date
    """)
)


#adding useful columns for analytics purpose
df = (
    
    # Surrogate key at month grain
    df.withColumn("date_key", f.date_format("month_start_date", "yyyyMM").cast("int"))
    .withColumn("year", f.year("month_start_date"))
    .withColumn("month_name", f.date_format("month_start_date", "MMMM"))
    .withColumn("month_short_name", f.date_format("month_start_date", "MMM"))
    .withColumn("quarter", f.concat(f.lit("Q"), f.quarter("month_start_date")))
    .withColumn("year_quarter", f.concat(f.col("year"), f.lit("-Q"), f.quarter("month_start_date")))
)


In [0]:
display(df)

In [0]:
# Save as a table
df.write \
    .mode("overwrite") \
    .format("delta") \
    .saveAsTable("fmcg.gold.dim_date")